In [1]:
import pickle
import pandas as pd

In [2]:
def pickle_load(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

    
def load(t3f_cpu_path, t3f_gpu_path, ttpy_path=None):
    cpu = pickle_load(t3f_cpu_path)
    gpu = pickle_load(t3f_gpu_path)
    d = {}
    op_list = ['matvec', 'matmul', 'norm', 'round', 'gram', 'project_rank100']
    
    one_cpu = []
    for op in op_list:
        if op == 'gram':
            # Single element Gram matrix is flat inner.
            curr_time = cpu['flatinner_time'] * 1000
        else:
            curr_time = cpu[op + '_time'] * 1000
        one_cpu.append(curr_time)
    d['one on CPU'] = one_cpu
    
    one_gpu = []
    for op in op_list:
        if op == 'gram':
            # Single element Gram matrix is flat inner.
            curr_time = gpu['flatinner_time'] * 1000
        else:
            curr_time = gpu[op + '_time'] * 1000
        one_gpu.append(curr_time)
    d['one on GPU'] = one_gpu
    
    batch_cpu = []
    for op in op_list:
        if op == 'gram':
            curr_time = cpu['batch_gram_time'] * 1000 / 100**2
        else:
            curr_time = cpu['batch_' + op + '_time'] * 1000 / 100
        batch_cpu.append(curr_time)
    d['batch on CPU'] = batch_cpu
    
    batch_gpu = []
    for op in op_list:
        if op == 'gram':
            curr_time = gpu['batch_gram_time'] * 1000 / 100.**2
        else:
            curr_time = gpu['batch_' + op + '_time'] * 1000 / 100.
#         curr_time = gpu['batch_' + op + '_time'] * 1000
        batch_gpu.append(curr_time)
    d['batch on GPU'] = batch_gpu
    
    if ttpy_path:
        ttpy = pickle_load(ttpy_path)
        ttpy_col = []
        for op in op_list:
            if op == 'gram':
                # Single element Gram matrix is flat inner.
                curr_time = ttpy['flatinner_time'] * 1000
            else:
                curr_time = ttpy[op + '_time'] * 1000
            ttpy_col.append(curr_time)
    d['ttpy, one on CPU'] = ttpy_col
    
    df = pd.DataFrame(d, index=op_list)
    regime_list = ['one on CPU', 'one on GPU', 'batch on CPU', 'batch on GPU']
    if ttpy_path:
        regime_list = ['ttpy, one on CPU'] + regime_list
    df = df[regime_list]
    return df

In [3]:
df = load('logs_py2_vkhrulkov_hatchiko2_cpu.pkl', 'logs_py2_vkhrulkov_hatchiko2_gpu.pkl', ttpy_path='logs_ttpy.pkl')

In [7]:
df.round(decimals=3)

,"ttpy, one on CPU",one on CPU,one on GPU,batch on CPU,batch on GPU
matvec,11.142,0.129,0.121,0.003,0.003
matmul,86.191,0.125,0.133,0.004,0.004
norm,3.790,1.902,0.893,0.422,0.050
round,73.027,0.159,0.165,0.006,0.006
gram,0.145,0.806,0.703,0.029,0.001
project_rank100,116.868,1.564,1.658,0.017,0.018


In [8]:
print(df.round(decimals=3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  ttpy, one on CPU &  one on CPU &  one on GPU &  batch on CPU &  batch on GPU \\
\midrule
matvec          &            11.142 &       0.129 &       0.121 &         0.003 &         0.003 \\
matmul          &            86.191 &       0.125 &       0.133 &         0.004 &         0.004 \\
norm            &             3.790 &       1.902 &       0.893 &         0.422 &         0.050 \\
round           &            73.027 &       0.159 &       0.165 &         0.006 &         0.006 \\
gram            &             0.145 &       0.806 &       0.703 &         0.029 &         0.001 \\
project\_rank100 &           116.868 &       1.564 &       1.658 &         0.017 &         0.018 \\
\bottomrule
\end{tabular}

